In [3]:
# %config InlineBackend.figure_format = "retina"
%config InlineBackend.figure_formats = ["pdf", "svg"]
from preamble import *

### 전처리

In [4]:
df_train = pd.read_csv('data/ratings_train.txt', 
                        delimiter="\t", 
                        keep_default_na=False) # 전처리할 시간 없어서 na값 우선 없애자
df_test = pd.read_csv('data/ratings_test.txt', 
                        delimiter="\t", 
                        keep_default_na=False)

In [5]:
text_train, y_train = df_train['document'].values, df_train['label'].values # 기본은 series로 반환 근데 특징추출하려면 시리즈 X 그래서 .values
text_test, y_test = df_test['document'].values, df_test['label'].values 
print(len(text_train), len(text_test))

150000 50000


In [6]:
# from sklearn.ensemble import RandomForestClassifier

# rf = RandomForestClassifier()
# rf.fit(text_train, y_train)
# rf.score(text_test, y_test)
# # 결과 : could not convert string to float: '아 더빙.. 진짜 짜증나네요 목소리' 
# # 한국어는 float으로 바꾸는 법이 없다(영어는 돼) -> 그래서 특징추출을 해줘야 함 - 한국어를 숫자로 바꾸자 - 그럼 어떻게?

### 특징추출

In [6]:
%pip install konlpy

Note: you may need to restart the kernel to use updated packages.


코퍼스 = 말뭉치
BOW : count

In [7]:
from konlpy.tag import Okt
okt = Okt()
ko_text = '한국어 분석을 시작해봐요!'
morphs = okt.morphs(ko_text)
pos = okt.pos(ko_text)
nouns = okt.nouns(ko_text)
print(f'형태소 : {morphs}')
print(f'품사 : {pos}')
print(f'nouns : {nouns}')


형태소 : ['한국어', '분석', '을', '시작', '해봐요', '!']
품사 : [('한국어', 'Noun'), ('분석', 'Noun'), ('을', 'Josa'), ('시작', 'Noun'), ('해봐요', 'Verb'), ('!', 'Punctuation')]
nouns : ['한국어', '분석', '시작']


In [8]:
okt = Okt()
def okt_tokenizer(text):
  return okt.morphs(text)

max_df : 전체에서 90% 이상 있으면 버려라 

In [16]:
from sklearn.feature_extraction.text import CountVectorizer # vectorize : 양자화 / 양으로 샐 수 없는 것을 양(量)으로 바꾼다
# 행위를 전달해줘야 함 Okt() 근데 이건 객체임 그래서 함수를 만들어줘야 해 그럼 필요할때 불러쓸거임 매개변수도 알아서 넣음
# 아무말 없으면 지가 가지고 있는 영문 tokenizer를 사용함
vectorizer = CountVectorizer(tokenizer=okt_tokenizer, 
                             token_pattern=None,
                             min_df=5,
                             max_df=0.9) 
X_train = vectorizer.fit_transform(text_train) # 데이터의 인식을 여기서 처음함 그래서 그 이전에 tokenizer를 지정해줘야해 default는 영문으로 되어 있다
X_test = vectorizer.fit_transform(text_test)  # 훈련데이터 score : 0.57...

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=okt_tokenizer, 
                             token_pattern=None,
                             ngram_range=(1,2),  # ngram 적용 전 : 0.6427048972941163 적용 후 : 0.8147203124130287
                             min_df=5,
                             max_df=0.9) 
X_train = vectorizer.fit_transform(text_train)
X_test = vectorizer.fit_transform(text_test)

In [23]:
# from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression

# tree = DecisionTreeClassifier()
# tree.fit(X_train, y_train)
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_train, y_train)

0.8147203124130287